In [2]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession

In [27]:
spark = pyspark.sql.SparkSession\
        .builder \
        .master("yarn") \
        .appName("ch07") \
        .config("spark.submit.deployMode", "client") \
        .config("spark.executor.memory", "2g") \
        .config("spark.driver.memory", "2g") \
        .getOrCreate()

25/05/13 13:06:07 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [28]:
spark.sparkContext

<SparkContext master=yarn appName=ch07>

In [29]:
df = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("hdfs:///data/retail-data/all/*.csv")


In [34]:
df.createOrReplaceTempView("retail_data")

In [35]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [95]:
from pyspark.sql.functions import col, expr, desc, asc

#### count

In [36]:
df.count()

541909

In [37]:
from pyspark.sql.functions import count

df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [38]:
spark.sql("""
    select count(*) 
    from retail_data
""").show()

+--------+
|count(1)|
+--------+
|  541909|
+--------+



#### countDistinct

In [39]:
from pyspark.sql.functions import countDistinct

df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [40]:
spark.sql(
"""
    select count(distinct(StockCode))
    from retail_data 
"""
).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



#### approx_count_distinct

In [41]:
from pyspark.sql.functions import approx_count_distinct

df.select(approx_count_distinct("StockCode", 0.1)).show()


+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [49]:
spark.sql("""
    select approx_count_distinct(StockCode, 0.1)
    from retail_data
""").show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



#### first and last

In [46]:
from pyspark.sql.functions import first, last

df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



In [47]:
spark.sql(
"""
    select first(StockCode), last(StockCode)
    from retail_data
"""
).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|           21544|         85049D|
+----------------+---------------+



#### min and max

In [50]:
from pyspark.sql.functions import min, max

df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [51]:
spark.sql("""
    select min(Quantity), max(Quantity)
    from retail_data
""").show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



#### sum

In [52]:
from pyspark.sql.functions import sum

df.select(sum("Quantity")).show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [53]:
spark.sql("""
    SELECT SUM(Quantity)
    FROM retail_data
""").show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



#### sumDistinct

In [55]:
from pyspark.sql.functions import sum_distinct

df.select(sum_distinct("Quantity")).show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [58]:
spark.sql("""
    select sum(distinct(Quantity))
    from retail_data
""").show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



#### Avg

In [59]:
from pyspark.sql.functions import avg, expr

df.select(
    count("Quantity"),
    sum("Quantity"),
    avg("Quantity"),
    expr("avg(Quantity)")
).show()

+---------------+-------------+----------------+----------------+
|count(Quantity)|sum(Quantity)|   avg(Quantity)|   avg(Quantity)|
+---------------+-------------+----------------+----------------+
|         541909|      5176450|9.55224954743324|9.55224954743324|
+---------------+-------------+----------------+----------------+



In [62]:
spark.sql("""
    select count(Quantity), sum(Quantity),
          avg(Quantity)
    from retail_data      
""").show()

+---------------+-------------+----------------+
|count(Quantity)|sum(Quantity)|   avg(Quantity)|
+---------------+-------------+----------------+
|         541909|      5176450|9.55224954743324|
+---------------+-------------+----------------+



#### Variance and Standard Deviation

In [64]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp

df.select(
    var_samp("Quantity"),
    stddev_samp("Quantity"),
    var_pop("Quantity"),
    stddev_pop("Quantity")
).show()

+------------------+---------------------+------------------+--------------------+
|var_samp(Quantity)|stddev_samp(Quantity)| var_pop(Quantity)|stddev_pop(Quantity)|
+------------------+---------------------+------------------+--------------------+
| 47559.39140929905|   218.08115785023486|47559.303646609354|  218.08095663447864|
+------------------+---------------------+------------------+--------------------+



In [68]:
spark.sql("""
    select var_samp(Quantity),
        stddev_samp(Quantity),
        var_pop(Quantity),
        stddev_pop(Quantity)
    from retail_data
""").show()

+------------------+---------------------+------------------+--------------------+
|var_samp(Quantity)|stddev_samp(Quantity)| var_pop(Quantity)|stddev_pop(Quantity)|
+------------------+---------------------+------------------+--------------------+
| 47559.39140929905|   218.08115785023486|47559.303646609354|  218.08095663447864|
+------------------+---------------------+------------------+--------------------+



#### skewness and kurtosis

In [66]:
from pyspark.sql.functions import skewness, kurtosis

df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610527843|119768.05495536518|
+--------------------+------------------+



In [69]:
spark.sql("""
    select skewness(Quantity), 
    kurtosis(Quantity)
    from retail_data
""").show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610527843|119768.05495536518|
+--------------------+------------------+



#### covariance and correlation

In [67]:
from pyspark.sql.functions import corr, covar_samp, covar_pop

df.select(
    corr("InvoiceNo", "Quantity"),
    covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity")
).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085637639E-4|             1052.7280543913773|            1052.7260778752732|
+-------------------------+-------------------------------+------------------------------+



In [70]:
spark.sql("""
    select
        corr(InvoiceNo, Quantity),
        covar_samp(InvoiceNo, Quantity),
        covar_pop(InvoiceNo, Quantity)
    from retail_data
""").show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085637639E-4|             1052.7280543913773|            1052.7260778752732|
+-------------------------+-------------------------------+------------------------------+



#### Aggregation Complex Types

In [74]:
from pyspark.sql.functions import collect_set, collect_list

df.select(collect_set("Country"), collect_list("Country")).show(1)

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [75]:
df.groupBy("CustomerID").agg(
    collect_list("Country"),
    collect_set("Country") 
).show(3)

+----------+---------------------+--------------------+
|CustomerID|collect_list(Country)|collect_set(Country)|
+----------+---------------------+--------------------+
|      NULL| [United Kingdom, ...|[France, Portugal...|
|     12346| [United Kingdom, ...|    [United Kingdom]|
|     12347| [Iceland, Iceland...|           [Iceland]|
+----------+---------------------+--------------------+
only showing top 3 rows



In [77]:
spark.sql("""
    select  collect_list(Country),
    collect_set(Country)
    from retail_data 
""").show(3)

+---------------------+--------------------+
|collect_list(Country)|collect_set(Country)|
+---------------------+--------------------+
| [United Kingdom, ...|[Portugal, Italy,...|
+---------------------+--------------------+



####  groupBy

In [79]:
df.groupBy("InvoiceNo", "CustomerID").count().show(5)

+---------+----------+-----+
|InvoiceNo|CustomerID|count|
+---------+----------+-----+
|   563017|     13198|   32|
|   563214|     16370|   71|
|   563372|     15653|    4|
|   563714|     14565|   31|
|   564666|     12492|    9|
+---------+----------+-----+
only showing top 5 rows



In [80]:
spark.sql("""
    select InvoiceNo, CustomerID, count(*)
    from retail_data
    group by InvoiceNo, CustomerID
""").show(5)

+---------+----------+--------+
|InvoiceNo|CustomerID|count(1)|
+---------+----------+--------+
|   563017|     13198|      32|
|   563214|     16370|      71|
|   563372|     15653|       4|
|   563714|     14565|      31|
|   564666|     12492|       9|
+---------+----------+--------+
only showing top 5 rows



In [82]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"), expr("stddev_pop(Quantity)")).show(5)

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
+---------+------------------+--------------------+
only showing top 5 rows



In [83]:
spark.sql("""
    select InvoiceNo, avg(Quantity), stddev_pop(Quantity)
    from retail_data
    group by InvoiceNo
""").show(5)

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   563020|16.041666666666668|  10.212244148841895|
|   565747|              10.3|   3.163858403911275|
|   566248|               9.0|   8.703447592764606|
|   566431| 14.11111111111111|   6.911254017815104|
|   567163|              14.5|  11.280514172678478|
+---------+------------------+--------------------+
only showing top 5 rows



#### Window Function

In [93]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, dense_rank, row_number, first_value, last_value
from pyspark.sql.functions import lag, lead, desc_nulls_last

In [96]:
df.where(col("CustomerID").isNotNull()).select("CustomerID",
    (row_number().over(Window.partitionBy("CustomerID").orderBy(desc_nulls_last("InvoiceNo"))).alias("row_number"))
).show(5)

+----------+----------+
|CustomerID|row_number|
+----------+----------+
|     12346|         1|
|     12346|         2|
|     12347|         1|
|     12347|         2|
|     12347|         3|
+----------+----------+
only showing top 5 rows



In [100]:
spark.sql("""
    select CustomerId,
          row_number() over(partition by CustomerID order by InvoiceNo) as row_number
    from retail_data
    where CustomerID is not null
""").show(5)

+----------+----------+
|CustomerId|row_number|
+----------+----------+
|     12346|         1|
|     12346|         2|
|     12347|         1|
|     12347|         2|
|     12347|         3|
+----------+----------+
only showing top 5 rows



In [140]:
from pyspark.sql.functions import to_date

datedf = df.withColumn("date", to_date("InvoiceDate", "MM/d/yyyy H:mm"))
datedf.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+--------------+---------+----------+-

In [126]:
datedf.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- date: date (nullable = true)



In [155]:
datedf.select("*").dtypes

[('InvoiceNo', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'int'),
 ('InvoiceDate', 'string'),
 ('UnitPrice', 'double'),
 ('CustomerID', 'int'),
 ('Country', 'string'),
 ('date', 'date')]

In [157]:
windowSpec = Window.partitionBy("CustomerID", "Country") \
    .orderBy(desc("Quantity")) \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

datedf.where("CustomerID is not NULL and  Quantity IS NOT NULL") \
      .orderBy("CustomerID") \
      .select(
          "CustomerID", "Country",
          (max("Quantity").over(windowSpec)).alias("max_purchase"),
          (rank().over(windowSpec)).alias("rank"),
          (dense_rank().over(windowSpec)).alias("dense_rank")
    ).show(5)

+----------+--------------+------------+----+----------+
|CustomerID|       Country|max_purchase|rank|dense_rank|
+----------+--------------+------------+----+----------+
|     12346|United Kingdom|       74215|   1|         1|
|     12346|United Kingdom|       74215|   2|         2|
|     12347|       Iceland|         240|   1|         1|
|     12347|       Iceland|         240|   2|         2|
|     12347|       Iceland|         240|   3|         3|
+----------+--------------+------------+----+----------+
only showing top 5 rows



In [166]:
spark.sql("""
    select CustomerID, Country, Quantity,
    max(Quantity) over(partition by CustomerID, Country order by Quantity ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as max_qty,
    rank(Quantity) over(partition by CustomerID, Country order by Quantity ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as rnk,
    dense_rank(Quantity) over(partition by CustomerID, Country order by Quantity ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)as d_rnk
    from retail_data
    order by CustomerID desc nulls last, rnk
""").show(10)

+----------+--------------+--------+-------+---+-----+
|CustomerID|       Country|Quantity|max_qty|rnk|d_rnk|
+----------+--------------+--------+-------+---+-----+
|     18287|United Kingdom|       4|      4|  1|    1|
|     18287|United Kingdom|       4|      4|  1|    1|
|     18287|United Kingdom|       4|      4|  1|    1|
|     18287|United Kingdom|       4|      4|  1|    1|
|     18287|United Kingdom|       6|      6|  5|    2|
|     18287|United Kingdom|       6|      6|  5|    2|
|     18287|United Kingdom|       6|      6|  5|    2|
|     18287|United Kingdom|       6|      6|  5|    2|
|     18287|United Kingdom|       6|      6|  5|    2|
|     18287|United Kingdom|       6|      6|  5|    2|
+----------+--------------+--------+-------+---+-----+
only showing top 10 rows



#### Grouping Sets

In [170]:
df.filter("CustomerID is not null and Country is not NULL") \
    .groupBy("CustomerID", "Country") \
    .sum("Quantity") \
    .orderBy("CustomerID") \
    .show(5)

+----------+--------------+-------------+
|CustomerID|       Country|sum(Quantity)|
+----------+--------------+-------------+
|     12346|United Kingdom|            0|
|     12347|       Iceland|         2458|
|     12348|       Finland|         2341|
|     12349|         Italy|          631|
|     12350|        Norway|          197|
+----------+--------------+-------------+
only showing top 5 rows



In [175]:
spark.sql("""
    select CustomerID, Country, sum(Quantity)
    from retail_data
    group by CustomerID, Country
    order by CustomerID desc NULLS LAST
""").show(5)

+----------+--------------+-------------+
|CustomerID|       Country|sum(Quantity)|
+----------+--------------+-------------+
|     18287|United Kingdom|         1586|
|     18283|United Kingdom|         1397|
|     18282|United Kingdom|           98|
|     18281|United Kingdom|           54|
|     18280|United Kingdom|           45|
+----------+--------------+-------------+
only showing top 5 rows



#### Rollup and cube

For Understanding Lets do it on small data.

In [176]:
df_small = spark.createDataFrame(
    [("a", "foo", 1),
     ("a", "foo", 2),
     ("a", "bar", 2),
     ("a", "bar", 2)],
    ["id", "x", "y"])
df_small.createOrReplaceTempView("df_small")

df_small.show()

+---+---+---+
| id|  x|  y|
+---+---+---+
|  a|foo|  1|
|  a|foo|  2|
|  a|bar|  2|
|  a|bar|  2|
+---+---+---+



<pre>
cube("id", "x", "y") will return (), (id), (x), (y), (id, x), (id, y), (x, y), (id, x, y).
(All the possible grouping existent combinations.)

rollup("id", "x", "y") will only return 
() ->  no grouping 
(id) -> Grouping on (id)
(id, x), -> grouping on (id,x)
(id, x, y) -> grouping on (id, x ,y)
(Combinations which include the beginning of the provided sequence.)

groupBy("id", "x", "y") will only return (id, x, y) combination.
</pre>

In [178]:
# rollup
df_small.rollup("id", "x", "y").count().show()

+----+----+----+-----+
|  id|   x|   y|count|
+----+----+----+-----+
|NULL|NULL|NULL|    4|
|   a|NULL|NULL|    4|
|   a| bar|NULL|    2|
|   a| bar|   2|    2|
|   a| foo|NULL|    2|
|   a| foo|   1|    1|
|   a| foo|   2|    1|
+----+----+----+-----+



In [179]:
df_small.cube("id", "x", "y").count().show()

+----+----+----+-----+
|  id|   x|   y|count|
+----+----+----+-----+
|NULL|NULL|   2|    3|
|NULL|NULL|NULL|    4|
|   a|NULL|   2|    3|
|   a| foo|NULL|    2|
|   a| foo|   1|    1|
|   a|NULL|   1|    1|
|NULL| foo|NULL|    2|
|   a|NULL|NULL|    4|
|NULL|NULL|   1|    1|
|NULL| foo|   2|    1|
|NULL| foo|   1|    1|
|   a| foo|   2|    1|
|NULL| bar|NULL|    2|
|NULL| bar|   2|    2|
|   a| bar|NULL|    2|
|   a| bar|   2|    2|
+----+----+----+-----+



In [180]:
spark.stop()